### PySpark 설정

In [2]:
# [+] pyspark 임포트 및 SparkConf, SparkContext 생성하기
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster('local').setAppName('reuctrion-operations')
sc = SparkContext(conf=conf)

In [3]:
# [+] reduce() 는 각 파티션별로 수행됨
sc.parallelize([1, 2, 3, 4, 5]).reduce(lambda x, y: x + y)

15

In [4]:
# [+] reduce() 주의점: 파티션 구성에 따라 결과 값이 달라질 수 있음
# 예: lambda x, y: (x * 2) + y
sc.parallelize([1, 2, 3, 4]).reduce(lambda x, y: (x * 2) + y)

26

In [6]:
sc.parallelize([1, 2, 3, 4], 2).reduce(lambda x, y: (x * 2) + y) # 위와 결과 다름

18

In [10]:
# [+] fold(): reduce()와 유사하며, 초기값을 지정
sc.parallelize([1, 2, 3, 4]).fold(1, lambda x, y : x + y)

12

In [13]:
sc.parallelize([1, 2, 3, 4], 2).fold(1, lambda x, y : x + y)

13

In [14]:
sc.parallelize([1, 2, 3, 4], 2).fold(0, lambda x, y : x * y)

0

In [16]:
# [+] groupBy(): 인자 함수를 기준으로 그룹핑을 수행
res = sc.parallelize([1, 1, 2, 3, 5, 8]).groupBy(lambda x: x % 2).collect()

# 나머지가 0이거나 1이거나. 두 그룹으로 그룹핑

In [19]:
res # key-value형태인데 resultiterable형태이기 때문에 for문으로 꺼내오기

[(1, <pyspark.resultiterable.ResultIterable at 0x22ed0f20970>),
 (0, <pyspark.resultiterable.ResultIterable at 0x22ee07e1ee0>)]

In [16]:
# [+] groupBy() 결과 출력하기
for k, v in res:
    print(k, list(v))
# resultiterable을 list형태로 변환시켜줘야함

1 [1, 1, 3, 5]
0 [2, 8]


In [22]:
# reduce(), fold(): 입출력 RDD 형식이 같아야 함
# single value -> pair 생성 시도할 경우 오류 / collectAndServe에러
sc.parallelize([1, 2, 3, 4]).reduce(lambda x, y: (x + y, x * y))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 15.0 failed 1 times, most recent failure: Lost task 0.0 in stage 15.0 (TID 23) (LAPTOP-K5S3UUQF executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Spark\spark-3.2.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 619, in main
  File "C:\Spark\spark-3.2.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 611, in process
  File "C:\Spark\spark-3.2.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\Users\wlgus\anaconda3\lib\site-packages\pyspark\rdd.py", line 997, in func
    yield reduce(f, iterator, initial)
  File "C:\Users\wlgus\anaconda3\lib\site-packages\pyspark\util.py", line 74, in wrapper
    return f(*args, **kwarg

In [18]:
sc.parallelize([1, 2, 3, 4]).fold((0, 0), lambda x, y: (x[0] + y, x[1] + 1))

TypeError: unsupported operand type(s) for +: 'int' and 'tuple'

In [23]:
"""
    aggregate(): 입출력 형식 다른 RDD 에 대한 reduction을 수행
    - zeroValues: 파티션 초기 값
    - seqOp: 각 파티션에 대한 reduction 함수
    - combOp: 파티션 결과를 합치는 reduction 함수
""" 
# single value RDD -> K-V RDD
# reduction함수에선 안되는데 aggregate()하면 가능

seqOp = lambda x, y: (x[0] + y, x[1] + 1)
combOp = lambda x, y: (x[0] + y[0], x[1] + y[1])

In [28]:
# [+] aggregate() 수행
sc.parallelize([1, 2, 3, 4], 2).aggregate((0, 0), seqOp, combOp)

(10, 4)